# Assignment 2

# Input and functions

Import existing libraries

In [1]:
import numpy as np
import pandas as pd
import sklearn.linear_model
import sklearn.metrics
import scipy.sparse

Score function for binary cross entropy Loss

In [2]:
def score_yp(y, p):
    return sklearn.metrics.log_loss(y, p)

# All input

In [3]:
# for model training
training_week_1 = 84
training_week_2 = 85
training_week_3 = 86
training_week_4 = 87
training_week_5 = 88

# for validation
validation_week_1 = 85 
validation_week_2 = 86
validation_week_3 = 87
validation_week_4 = 88
validation_week_5 = 89

# for the final prediction
test_week = 90  

target_customers = list(range(2000)) # the first 2000 customers (id: 0-1999)
target_products = list(range(250))

My path

In [4]:
PATH_DATA = "C:\\Users\Lodew\Documents\Minor_LFBD\Module 2"

Import raw dataframes from canvas, filter the data and inspect the datasets 

In [5]:
coupons = pd.read_parquet(f"{PATH_DATA}\\coupons.parquet")
coupons = coupons[coupons["customer"] < 2000]

In [6]:
coupons

,week,customer,product,discount
0,0,0,35,35
1,0,0,193,40
2,0,0,27,30
3,0,0,177,35
4,0,0,5,30
...,...,...,...,...
44509995,89,1999,108,10
44509996,89,1999,207,25
44509997,89,1999,246,30
44509998,89,1999,51,15


In [7]:
coupons.describe().astype(int)

,week,customer,product,discount
count,900000,900000,900000,900000
mean,44,999,124,24
std,25,577,72,9
min,0,0,0,10
25%,22,499,62,15
50%,44,999,125,25
75%,67,1499,187,35
max,89,1999,249,40


In [8]:
coupons = coupons.drop("discount", True)

C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3411537383.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  coupons = coupons.drop("discount", True)


In [9]:
coupons.describe().astype(int)

,week,customer,product
count,900000,900000,900000
mean,44,999,124
std,25,577,72
min,0,0,0
25%,22,499,62
50%,44,999,125
75%,67,1499,187
max,89,1999,249


In [10]:
baskets = pd.read_parquet(f"{PATH_DATA}\\baskets.parquet")
baskets_with_price = baskets[baskets["customer"] < 2000]
baskets = baskets[baskets["customer"] < 2000].drop("price", True)

C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/67737455.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  baskets = baskets[baskets["customer"] < 2000].drop("price", True)


In [11]:
baskets_with_price

,week,customer,product,price
0,0,0,71,629
1,0,0,91,605
2,0,0,116,715
3,0,0,123,483
4,0,0,157,592
...,...,...,...,...
68091869,89,1999,144,562
68091870,89,1999,158,566
68091871,89,1999,192,549
68091872,89,1999,213,592


In [12]:
baskets.describe().astype(int)

,week,customer,product
count,1378720,1378720,1378720
mean,44,997,125
std,25,576,69
min,0,0,0
25%,22,499,66
50%,44,993,123
75%,67,1496,189
max,89,1999,249


In [13]:
prediction = pd.read_parquet(f"{PATH_DATA}\\prediction_index.parquet")

# Deriving category-product features using co-occurence matrix

In [ ]:
baskets["basket"] = baskets["customer"] * 1000000 + baskets["week"]

Function for making a co-occurence matrix

In [ ]:
def get_co_occurrences(x, variable_basket="basket", variable_product="product"):
    row = x[variable_basket].values
    col = x[variable_product].values
    dim = (x[variable_basket].max() + 1, x[variable_product].max() + 1)

    basket_product_table = scipy.sparse.csr_matrix(
        (np.ones(len(row), dtype=int), (row, col)), shape=dim
    )
    co_occurrences_sparse = basket_product_table.T.dot(basket_product_table).tocoo()
    co_occurrences_df = pd.DataFrame(
        {
            "product_1": co_occurrences_sparse.row,
            "product_2": co_occurrences_sparse.col,
            "co-occurrence": co_occurrences_sparse.data,
        }
    )
    return co_occurrences_df

In [ ]:
co_occurrences = get_co_occurrences(baskets)

In [ ]:
co_occurrences

Check whether the first 10 (0 - 9) products occur with each other. 
We see that these products only occur with themselves and therefore we can assume that these are substitutes from each other and will be in the same category.

In [ ]:
co_occurrences[
    (co_occurrences["product_1"].isin(range(10)))
    & (co_occurrences["product_2"].isin(range(10)))
]

In [ ]:
co_occurrences[(co_occurrences["product_1"] == 0) & (co_occurrences["product_2"] == 1)]

Deriving category-product map

In [14]:
def build_category_product_map():

    category_product_map = pd.DataFrame(
        {
            "product": list(range(250)),
            "category": np.repeat(range(25), 10),
        }
    )
    
    return category_product_map

In [15]:
build_category_product_map()

,product,category
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
245,245,24
246,246,24
247,247,24
248,248,24


In [16]:
baskets = baskets.merge(build_category_product_map(), on="product")

In [17]:
baskets

,week,customer,product,category
0,0,0,71,7
1,0,47,71,7
2,0,55,71,7
3,0,96,71,7
4,0,110,71,7
...,...,...,...,...
1378715,89,1109,175,17
1378716,89,1150,175,17
1378717,89,1242,175,17
1378718,89,1523,175,17


In [18]:
def baseline_prediction(baskets, week):

    baskets_t = baskets[baskets["week"] < week].reset_index(drop=True)
    n_weeks = baskets_t.week.nunique()
    print(n_weeks)

    purchase_frequency_ij = (
        (baskets_t.groupby(["customer", "product"])[["week"]].count() / n_weeks)
        .rename(columns={"week": "probability"})
        .reset_index()
    )

    df = pd.DataFrame(
        {
            "week": week,
            "customer": np.repeat(target_customers, len(target_products), axis=0),
            "product": target_products * len(target_customers),
        }
    )

    result_baseline = pd.merge(
        df,
        purchase_frequency_ij,
        on=["customer", "product"],
        how="left",
    ).fillna(0)

    return result_baseline

In [27]:
baseline_prediction(baskets, test_week)

90


,week,customer,product,probability
0,90,0,0,0.000000
1,90,0,1,0.000000
2,90,0,2,0.000000
3,90,0,3,0.000000
4,90,0,4,0.088889
...,...,...,...,...
499995,90,1999,245,0.133333
499996,90,1999,246,0.000000
499997,90,1999,247,0.000000
499998,90,1999,248,0.000000


# Functions for building features

Function for building price feature

In [20]:
def build_product_price(baskets_with_price, week):
    
    baskets_product_price = baskets_with_price[baskets_with_price["week"] < week]
    df = pd.DataFrame({"product": list(range(250))})
    
    df = df.merge(baskets_product_price, on="product", how="left")
    
    price = (
        (
            df.groupby(["customer", "product"])[["price"]].max()
        ).rename(columns={"week": "price"})
        .reset_index()
    )
    
    return price 

In [21]:
build_product_price(baskets_with_price, training_week_1).sample(30)

,customer,product,price
32178,461,38,699
65818,944,226,536
93529,1341,28,442
2717,40,128,630
3011,44,106,478
87217,1249,239,493
124828,1791,194,546
79975,1145,88,471
3298,48,133,509
15835,227,204,496


Function for building category frequency feature

In [22]:
def build_category_frequency_feature(baskets, week_start, week_end, feature_name):
    
    baskets_subset = baskets[
        (baskets["week"] >= week_start) & (baskets["week"] <= week_end)
    ]

    category_frequency_ij = (
        (
            baskets_subset.groupby(["customer", "category"])[["week"]].count()
            / baskets_subset.week.nunique()
        )
        .rename(columns={"week": feature_name})
        .reset_index()
    )

    return category_frequency_ij

Function for building product purchase frequency features per customer

In [23]:
def build_purchase_frequency_feature(baskets, week_start, week_end, feature_name):
    
    baskets_subset = baskets[
        (baskets["week"] >= week_start) & (baskets["week"] <= week_end)
    ]
    print(baskets_subset.week.nunique())

    purchase_frequency_ij = (
        (
            baskets_subset.groupby(["customer", "product"])[["week"]].count()
            / baskets_subset.week.nunique()
        )
        .rename(columns={"week": feature_name})
        .reset_index()
    )

    return purchase_frequency_ij

In [24]:
def build_pruchase_frequency_norm_feature(baskets, week_start, week_end, feaure_name):
    
    category_product_map = build_category_product_map()
    
    purchase_frequency_ic = build_category_frequency_feature(baskets, week_start, week_end, "frequency_category")
    
    purchase_frequency_ij =  build_purchase_frequency_feature(baskets, week_start, week_end, "frequency_product")
    
    purchase_frequency_ijc = (
            purchase_frequency_ij.merge(category_product_map, on="product")
            .merge(purchase_frequency_ic, on=["customer", "category"], suffixes=["_product", "_category"])
    )
    
    purchase_frequency_ijc[feaure_name] = (
            purchase_frequency_ijc["frequency_product"] 
            / purchase_frequency_ijc["frequency_category"]
    )
    
    return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)

Function for building target dataframe

In [25]:
def build_target(baskets, week):

    baskets_week = baskets[baskets["week"] == week][
        ["week", "customer", "product"]
    ].reset_index(drop=True)
    baskets_week["y"] = 1

    df = pd.DataFrame(
        {
            "week": week,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
            "customer": np.repeat(target_customers, len(target_products), axis=0),
            "product": target_products * len(target_customers),
        }
    )

    df = df.merge(baskets_week, on=["week", "customer", "product"], how="left")
          
    df["y"] = df["y"].fillna(0).astype(int)
    
    df = df.merge(build_category_product_map(), on="product", how="left")
    
    return df

Function for building coupon received in last week feature

In [26]:
def build_coupon_received_last_week_feature(coupons, week):
    
    coupons_week = coupons[coupons['week'] == week][
        ["week", "customer", "product"]
    ].reset_index(drop=True)
    coupons_week["coupon_recieved_last_week"] = 1
    
    df = pd.DataFrame(
        {
            "week": week,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
            "customer": np.repeat(target_customers, len(target_products), axis=0),
            "product": target_products * len(target_customers),
        }
    )
    
    df = df.merge(coupons_week, on=["week", "customer", "product"], how="left")
    df["coupon_recieved_last_week"] = df["coupon_recieved_last_week"].fillna(0).astype(int)
    
    return df

Function for building coupon category frequency feature

In [28]:
def build_coupon_category_frequency_feature(coupons, week):
    
    coupons_train = coupons[coupons["week"] < week]
    n_weeks = coupons_train.week.nunique()
    coupons_train["coupon_recieved"] = 1
    
    coupons_train = coupons_train.merge(build_category_product_map(), on=["product"], how="left")

    coupons_ic = (
        (
            coupons_train.groupby(["customer", "category"])[["week"]].count()
            / n_weeks
        )
        .rename(columns={"week": "coupon_category_frequency"})
        .reset_index()
    )
    
    return coupons_ic

In [29]:
build_coupon_category_frequency_feature(coupons, training_week_1)

C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2415493297.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1


,customer,category,coupon_category_frequency
0,0,0,0.238095
1,0,1,0.214286
2,0,2,0.226190
3,0,3,0.273810
4,0,4,0.166667
...,...,...,...
49995,1999,20,0.297619
49996,1999,21,0.238095
49997,1999,22,0.142857
49998,1999,23,0.214286


Function for building coupon frequency features

In [30]:
def build_coupon_frequency_feature(coupons, week):
    
    coupons_train = coupons[coupons["week"] < week]
    n_weeks = coupons_train.week.nunique()
    coupons_train["coupon_recieved"] = 1

    coupons_ij = (
        (
            coupons_train.groupby(["customer", "product"])[["week"]].count()
            / n_weeks
        )
        .rename(columns={"week": "coupon_frequency"})
        .reset_index()
    )
    
    return coupons_ij

In [31]:
build_coupon_frequency_feature(coupons, training_week_1)

C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2261255535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1


,customer,product,coupon_frequency
0,0,1,0.023810
1,0,2,0.023810
2,0,3,0.011905
3,0,4,0.035714
4,0,5,0.023810
...,...,...,...
408261,1999,245,0.035714
408262,1999,246,0.023810
408263,1999,247,0.035714
408264,1999,248,0.023810


In [32]:
def build_coupon_frequency_norm_feature(baskets, coupons, week):
    
    base = build_target(baskets, week).drop("week", True)
    
    coupons_ij = build_coupon_frequency_feature(coupons, week)
    coupons_ic = build_coupon_category_frequency_feature(coupons, week)
    
    coupons_ijc = (
            base.merge(coupons_ij, on=["customer", "product"], how="left")
            .merge(coupons_ic, on=["customer", "category"], how="left").fillna(0)
    )
    

    coupons_ijc["coupon_frequency_norm"] = (
            coupons_ijc["coupon_frequency"] 
            / coupons_ijc["coupon_category_frequency"]
    )
    
    return coupons_ijc.drop(["coupon_frequency", "coupon_category_frequency"], True)

In [33]:
build_coupon_frequency_norm_feature(baskets, coupons, training_week_1)

C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  base = build_target(baskets, week).drop("week", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2261255535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return coupons_ijc.drop(["coupon_frequency", "coupon_category_frequency"], True)


,customer,product,y,category,coupon_frequency_norm
0,0,0,0,0,0.000000
1,0,1,0,0,0.100000
2,0,2,0,0,0.100000
3,0,3,0,0,0.050000
4,0,4,0,0,0.150000
...,...,...,...,...,...
499995,1999,245,0,24,0.130435
499996,1999,246,0,24,0.086957
499997,1999,247,0,24,0.130435
499998,1999,248,0,24,0.086957


Function for building recency feature (bought in last three weeks) per customer

In [34]:
def bought_in_last_three_weeks(baskets, week_start, week_end):
    
    baskets_subset = baskets[
        (baskets["week"] >= week_start) & (baskets["week"] <= week_end)
    ]
        
    recency_ij = (
        (
        (week_end) - baskets_subset.groupby(["customer", "product"])[["week"]].max()
        )
        .rename(columns={"week": "recency"})
        .reset_index()
    )
    
    recency_ij["bought_in_last_3_weeks"] = np.ones(len(recency_ij)).astype(int)
    
    
    return recency_ij.drop("recency", True)  

Function for building recency feature (bought in last ten weeks but not in the last three weeks) per customer

In [35]:
def bought_in_last_ten_weeks(baskets, week_start, week_end):
    
    baskets_subset = baskets[
        (baskets["week"] >= week_start) & (baskets["week"] <= week_end)
    ]
    
        
    recency_ij = (
        (
        (week_end) - baskets_subset.groupby(["customer", "product"])[["week"]].max()
        )
        .rename(columns={"week": "recency"})
        .reset_index()
    )
    
    recency_ij["bought_in_last_10_weeks"] = np.ones(len(recency_ij)).astype(int)
    
    return recency_ij.drop("recency", True)

In [36]:
def bought_in_last_ten_weeks_correct(baskets, week_start, week_end):
    
    last_3_weeks = bought_in_last_three_weeks(baskets, week_start + 7, week_end + 3)
    last_10_weeks = bought_in_last_ten_weeks(baskets, week_start, week_end)
    
    df = (
        pd.merge(last_10_weeks, last_3_weeks, 
        on=["customer", "product"], how="left")
        .fillna(0).astype(int)
    )
    
    df["bought_in_last_10_weeks"] = df["bought_in_last_10_weeks"] - df["bought_in_last_3_weeks"]
    
    return df.drop("bought_in_last_3_weeks", True)

Function for building base table with all features

In [40]:
def build_base_table(baskets, baskets_with_price, coupons, week):
    
    # target
    y = build_target(baskets, week)
    
    # features
    x_0 = build_product_price(baskets_with_price, week)
    x_1 = build_purchase_frequency_feature(baskets, -1, week - 1, "frequency_full")
    x_2 = build_purchase_frequency_feature(baskets, week - 30, week - 1, "frequency_l30")
    x_3 = build_purchase_frequency_feature(baskets, week - 5, week - 1, "frequency_l5")
    x_4 = build_category_frequency_feature(baskets, -1, week - 1, "frequency_category_full")
    x_5 = build_category_frequency_feature(baskets, week - 30, week - 1, "frequency_category_l30")
    x_6 = build_category_frequency_feature(baskets, week - 5, week - 1, "frequency_category_l5")
    x_7 = build_pruchase_frequency_norm_feature(baskets, -1, week - 1, "frequency_full_norm")
    x_8 = build_pruchase_frequency_norm_feature(baskets, week - 30, week - 1, "frequency_l30_norm")
    x_9 = build_pruchase_frequency_norm_feature(baskets, week - 5, week - 1, "frequency_l5_norm")
    x_10 = build_coupon_frequency_feature(coupons, week)
    x_11 = build_coupon_category_frequency_feature(coupons, week)
    x_12 = build_coupon_frequency_norm_feature(baskets, coupons, week)
    x_13 = bought_in_last_three_weeks(baskets, week - 3, week - 1)
    x_14 = bought_in_last_ten_weeks_correct(baskets, week - 10, week - 4)
    x_15 = build_coupon_received_last_week_feature(coupons, week)
    
    base_table_yx = (
        y.merge(x_0, on=["customer", "product"], how="left")
        .merge(x_1, on=["customer", "product"], how="left")
        .merge(x_2, on=["customer", "product"], how="left")
        .merge(x_3, on=["customer", "product"], how="left")
        .merge(x_4, on=["customer", "category"], how="left")
        .merge(x_5, on=["customer", "category"], how="left")
        .merge(x_6, on=["customer", "category"], how="left")
        .merge(x_7, on=["customer", "product", "category"], how="left")
        .merge(x_8, on=["customer", "product", "category"], how="left")
        .merge(x_9, on=["customer", "product", "category"], how="left")
        .merge(x_10, on=["customer", "product"], how="left")
        .merge(x_11, on=["customer", "category"], how="left")
        .merge(x_12, on=["customer", "product", "category", "y"], how="left")
        .merge(x_13, on=["customer", "product"], how="left")
        .merge(x_14, on=["customer", "product"], how="left")
        .merge(x_15, on=["week", "customer", "product"], how="left")
        .fillna(0)
    )
    
    return base_table_yx

# Training data and validating model for 5 different weeks

# 1. Week 85

Training: Train model (week < 85) 

In [41]:
base_table_train = build_base_table(baskets, baskets_with_price, coupons, training_week_1)

84
30
5
84


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_values except for the argument 'by' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)


30
5


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2261255535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  base = build_target(baskets, week).drop("week", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/727766109.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return recency_ij.drop("recency", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/1234096421.py:14: FutureWarning: In a future version of pandas

In [42]:
base_table_train.sample(30)

,week,customer,product,y,category,price,frequency_full,frequency_l30,frequency_l5,frequency_category_full,...,frequency_category_l5,frequency_full_norm,frequency_l30_norm,frequency_l5_norm,coupon_frequency,coupon_category_frequency,coupon_frequency_norm,bought_in_last_3_weeks,bought_in_last_10_weeks,coupon_recieved_last_week
199876,84,799,126,0,12,598.0,0.011905,0.000000,0.0,0.345238,...,0.8,0.034483,0.000000,0.000000,0.000000,0.238095,0.000000,0.0,0.0,0
304657,84,1218,157,0,15,0.0,0.000000,0.000000,0.0,0.214286,...,0.4,0.000000,0.000000,0.000000,0.011905,0.190476,0.062500,0.0,0.0,0
163259,84,653,9,0,0,759.0,0.011905,0.000000,0.0,0.297619,...,0.2,0.040000,0.000000,0.000000,0.011905,0.166667,0.071429,0.0,0.0,0
289882,84,1159,132,1,13,646.0,0.642857,0.566667,0.6,0.928571,...,1.0,0.692308,0.607143,0.600000,0.000000,0.202381,0.000000,1.0,0.0,0
344142,84,1376,142,0,14,0.0,0.000000,0.000000,0.0,0.595238,...,0.4,0.000000,0.000000,0.000000,0.035714,0.178571,0.200000,0.0,0.0,0
133578,84,534,78,0,7,625.0,0.166667,0.166667,0.4,0.607143,...,0.6,0.274510,0.263158,0.666667,0.023810,0.345238,0.068966,1.0,0.0,0
398334,84,1593,84,0,8,501.0,0.226190,0.166667,0.4,0.369048,...,0.4,0.612903,0.454545,1.000000,0.000000,0.202381,0.000000,1.0,0.0,0
235218,84,940,218,0,21,0.0,0.000000,0.000000,0.0,0.357143,...,0.4,0.000000,0.000000,0.000000,0.023810,0.190476,0.125000,0.0,0.0,0
115476,84,461,226,0,22,536.0,0.095238,0.066667,0.0,0.452381,...,0.4,0.210526,0.166667,0.000000,0.047619,0.226190,0.210526,0.0,1.0,0
362767,84,1451,17,0,1,0.0,0.000000,0.000000,0.0,0.309524,...,0.2,0.000000,0.000000,0.000000,0.011905,0.154762,0.076923,0.0,0.0,0


In [43]:
features = ["price",
            "frequency_full", "frequency_l30", "frequency_l5",
            "frequency_category_full", "frequency_category_l30", "frequency_category_l5",
            "frequency_full_norm", "frequency_l30_norm", "frequency_l5_norm", 
            "coupon_frequency", "coupon_category_frequency",
            "bought_in_last_3_weeks", "bought_in_last_10_weeks",
            "coupon_recieved_last_week"
]
y = base_table_train["y"].values
X = base_table_train[features].values

In [44]:
log_reg = sklearn.linear_model.LogisticRegression(max_iter=300).fit(X, y)

In [45]:
log_reg.intercept_, log_reg.coef_

(array([-6.19578231]),
 array([[ 0.00293311,  2.33985436,  1.61880501, -0.03583269,  1.34737043,
          0.76242034, -0.90610745,  1.28051764,  0.33363958, -0.16226675,
         -0.11594866, -1.02499933,  0.78637443,  0.78565995,  1.28315762]]))

In [46]:
base_table_train["probability"] = log_reg.predict_proba(X)[:, 1]

In [47]:
score_yp(
    base_table_train["y"].values,
    base_table_train["probability"].values
)

0.07959287892755136

Validation: Test model performance (week 85)

In [48]:
base_table_validation = build_base_table(baskets, baskets_with_price, coupons, validation_week_1)

85
30
5
85


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_values except for the argument 'by' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)


30
5


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2261255535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  base = build_target(baskets, week).drop("week", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/727766109.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return recency_ij.drop("recency", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/1234096421.py:14: FutureWarning: In a future version of pandas

In [49]:
base_table_validation

,week,customer,product,y,category,price,frequency_full,frequency_l30,frequency_l5,frequency_category_full,...,frequency_category_l5,frequency_full_norm,frequency_l30_norm,frequency_l5_norm,coupon_frequency,coupon_category_frequency,coupon_frequency_norm,bought_in_last_3_weeks,bought_in_last_10_weeks,coupon_recieved_last_week
0,85,0,0,0,0,0.0,0.000000,0.000000,0.0,0.094118,...,0.0,0.000000,0.000000,0.0,0.000000,0.235294,0.000000,0.0,0.0,0
1,85,0,1,0,0,0.0,0.000000,0.000000,0.0,0.094118,...,0.0,0.000000,0.000000,0.0,0.023529,0.235294,0.100000,0.0,0.0,0
2,85,0,2,0,0,0.0,0.000000,0.000000,0.0,0.094118,...,0.0,0.000000,0.000000,0.0,0.023529,0.235294,0.100000,0.0,0.0,0
3,85,0,3,0,0,0.0,0.000000,0.000000,0.0,0.094118,...,0.0,0.000000,0.000000,0.0,0.011765,0.235294,0.050000,0.0,0.0,0
4,85,0,4,0,0,620.0,0.094118,0.100000,0.0,0.094118,...,0.0,1.000000,1.000000,0.0,0.035294,0.235294,0.150000,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,85,1999,245,0,24,549.0,0.129412,0.133333,0.0,0.635294,...,0.8,0.203704,0.190476,0.0,0.035294,0.270588,0.130435,0.0,0.0,1
499996,85,1999,246,0,24,0.0,0.000000,0.000000,0.0,0.635294,...,0.8,0.000000,0.000000,0.0,0.023529,0.270588,0.086957,0.0,0.0,0
499997,85,1999,247,0,24,0.0,0.000000,0.000000,0.0,0.635294,...,0.8,0.000000,0.000000,0.0,0.035294,0.270588,0.130435,0.0,0.0,0
499998,85,1999,248,0,24,0.0,0.000000,0.000000,0.0,0.635294,...,0.8,0.000000,0.000000,0.0,0.023529,0.270588,0.086957,0.0,0.0,0


In [50]:
features = ["price",
            "frequency_full", "frequency_l30", "frequency_l5",
            "frequency_category_full", "frequency_category_l30", "frequency_category_l5",
            "frequency_full_norm", "frequency_l30_norm", "frequency_l5_norm", 
            "coupon_frequency", "coupon_category_frequency",
            "bought_in_last_3_weeks", "bought_in_last_10_weeks",
            "coupon_recieved_last_week"
]
X_validation = base_table_validation[features].values

In [51]:
base_table_validation["probability"] = log_reg.predict_proba(X_validation)[:, 1]

In [52]:
score_yp(
    base_table_validation["y"].values,
    base_table_validation["probability"].values
)

0.08033753358067258

# 2. Week 86

Training: Train model (week < 86) 

In [53]:
base_table_train = build_base_table(baskets, baskets_with_price, coupons, training_week_2)

85
30
5
85


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_values except for the argument 'by' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)


30
5


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2261255535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  base = build_target(baskets, week).drop("week", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/727766109.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return recency_ij.drop("recency", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/1234096421.py:14: FutureWarning: In a future version of pandas

In [54]:
features = ["price",
            "frequency_full", "frequency_l30", "frequency_l5",
            "frequency_category_full", "frequency_category_l30", "frequency_category_l5",
            "frequency_full_norm", "frequency_l30_norm", "frequency_l5_norm", 
            "coupon_frequency", "coupon_category_frequency",
            "bought_in_last_3_weeks", "bought_in_last_10_weeks",
            "coupon_recieved_last_week"
]
y = base_table_train["y"].values
X = base_table_train[features].values

In [55]:
log_reg = sklearn.linear_model.LogisticRegression(max_iter=300).fit(X, y)

In [56]:
log_reg.intercept_, log_reg.coef_

(array([-6.30986884]),
 array([[ 0.00312161,  3.00881726,  1.27846013, -0.52902172,  1.86127665,
          0.47453313, -1.02922707,  1.73456496, -0.20017557,  0.04905759,
         -0.12234269, -0.80148259,  0.67554889,  0.71019741,  0.99465866]]))

In [57]:
base_table_train["probability"] = log_reg.predict_proba(X)[:, 1]

In [58]:
score_yp(
    base_table_train["y"].values,
    base_table_train["probability"].values
)

0.0801546402153619

Validation: Test model performance (week 86)

In [59]:
base_table_validation = build_base_table(baskets, baskets_with_price, coupons, validation_week_2)

86
30
5
86


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_values except for the argument 'by' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)


30
5


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2261255535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  base = build_target(baskets, week).drop("week", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/727766109.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return recency_ij.drop("recency", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/1234096421.py:14: FutureWarning: In a future version of pandas

In [60]:
features = ["price",
            "frequency_full", "frequency_l30", "frequency_l5",
            "frequency_category_full", "frequency_category_l30", "frequency_category_l5",
            "frequency_full_norm", "frequency_l30_norm", "frequency_l5_norm", 
            "coupon_frequency", "coupon_category_frequency",
            "bought_in_last_3_weeks", "bought_in_last_10_weeks",
            "coupon_recieved_last_week"
]
X_validation = base_table_validation[features].values

In [61]:
base_table_validation["probability"] = log_reg.predict_proba(X_validation)[:, 1]

In [62]:
score_yp(
    base_table_validation["y"].values,
    base_table_validation["probability"].values
)

0.08007736291773794

# 3. Week 87

Training: Train model (week < 87) 

In [63]:
base_table_train = build_base_table(baskets, baskets_with_price, coupons, training_week_3)

86
30
5
86


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_values except for the argument 'by' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)


30
5


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2261255535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  base = build_target(baskets, week).drop("week", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/727766109.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return recency_ij.drop("recency", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/1234096421.py:14: FutureWarning: In a future version of pandas

In [64]:
features = ["price",
            "frequency_full", "frequency_l30", "frequency_l5",
            "frequency_category_full", "frequency_category_l30", "frequency_category_l5",
            "frequency_full_norm", "frequency_l30_norm", "frequency_l5_norm", 
            "coupon_frequency", "coupon_category_frequency",
            "bought_in_last_3_weeks", "bought_in_last_10_weeks",
            "coupon_recieved_last_week"
]
y = base_table_train["y"].values
X = base_table_train[features].values

In [65]:
log_reg = sklearn.linear_model.LogisticRegression(max_iter=300).fit(X, y)

In [66]:
log_reg.intercept_, log_reg.coef_

(array([-6.14414522]),
 array([[ 0.00300536,  2.49492596,  1.65480085, -0.09239614,  1.4070975 ,
          0.7634536 , -1.03106258,  1.36502895,  0.17044367, -0.12583485,
         -0.13776793, -1.06092215,  0.73488541,  0.7046196 ,  1.24116761]]))

In [67]:
base_table_train["probability"] = log_reg.predict_proba(X)[:, 1]

In [68]:
score_yp(
    base_table_train["y"].values,
    base_table_train["probability"].values
)

0.08019282171379094

Validation: Test model performance (week 87)

In [69]:
base_table_validation = build_base_table(baskets, baskets_with_price, coupons, validation_week_3)

87
30
5
87


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_values except for the argument 'by' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)


30
5


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2261255535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  base = build_target(baskets, week).drop("week", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/727766109.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return recency_ij.drop("recency", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/1234096421.py:14: FutureWarning: In a future version of pandas

In [70]:
features = ["price",
            "frequency_full", "frequency_l30", "frequency_l5",
            "frequency_category_full", "frequency_category_l30", "frequency_category_l5",
            "frequency_full_norm", "frequency_l30_norm", "frequency_l5_norm", 
            "coupon_frequency", "coupon_category_frequency",
            "bought_in_last_3_weeks", "bought_in_last_10_weeks",
            "coupon_recieved_last_week"
]
X_validation = base_table_validation[features].values

In [71]:
base_table_validation["probability"] = log_reg.predict_proba(X_validation)[:, 1]

In [72]:
score_yp(
    base_table_validation["y"].values,
    base_table_validation["probability"].values
)

0.0804073225040108

# 4. Week 88

Training: Train model (week < 88) 

In [73]:
base_table_train = build_base_table(baskets, baskets_with_price, coupons, training_week_4)

87
30
5
87


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_values except for the argument 'by' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)


30
5


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2261255535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  base = build_target(baskets, week).drop("week", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/727766109.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return recency_ij.drop("recency", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/1234096421.py:14: FutureWarning: In a future version of pandas

In [74]:
features = ["price",
            "frequency_full", "frequency_l30", "frequency_l5",
            "frequency_category_full", "frequency_category_l30", "frequency_category_l5",
            "frequency_full_norm", "frequency_l30_norm", "frequency_l5_norm", 
            "coupon_frequency", "coupon_category_frequency",
            "bought_in_last_3_weeks", "bought_in_last_10_weeks",
            "coupon_recieved_last_week"
]
y = base_table_train["y"].values
X = base_table_train[features].values

In [75]:
log_reg = sklearn.linear_model.LogisticRegression(max_iter=300).fit(X, y)

In [76]:
log_reg.intercept_, log_reg.coef_

(array([-6.24618466]),
 array([[ 0.00303381,  2.3551254 ,  1.52887682, -0.11063528,  1.40070937,
          0.78001217, -0.93058328,  1.34198305,  0.34618307, -0.1211492 ,
         -0.10060364, -0.99324253,  0.76349743,  0.70949329,  1.22940245]]))

In [77]:
base_table_train["probability"] = log_reg.predict_proba(X)[:, 1]

In [78]:
score_yp(
    base_table_train["y"].values,
    base_table_train["probability"].values
)

0.08040011657289747

Validation: Test model performance (week 88)

In [79]:
base_table_validation = build_base_table(baskets, baskets_with_price, coupons, validation_week_4)

88
30
5
88


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_values except for the argument 'by' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)


30
5


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2261255535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  base = build_target(baskets, week).drop("week", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/727766109.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return recency_ij.drop("recency", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/1234096421.py:14: FutureWarning: In a future version of pandas

In [80]:
features = ["price",
            "frequency_full", "frequency_l30", "frequency_l5",
            "frequency_category_full", "frequency_category_l30", "frequency_category_l5",
            "frequency_full_norm", "frequency_l30_norm", "frequency_l5_norm", 
            "coupon_frequency", "coupon_category_frequency",
            "bought_in_last_3_weeks", "bought_in_last_10_weeks",
            "coupon_recieved_last_week"
]
X_validation = base_table_validation[features].values

In [81]:
base_table_validation["probability"] = log_reg.predict_proba(X_validation)[:, 1]

In [82]:
score_yp(
    base_table_validation["y"].values,
    base_table_validation["probability"].values
)

0.08034705904590832

# 5. Week 89

Training: Train model (week < 89) 

In [83]:
base_table_train = build_base_table(baskets, baskets_with_price, coupons, training_week_5)

88
30
5
88


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_values except for the argument 'by' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)


30
5


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2261255535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  base = build_target(baskets, week).drop("week", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/727766109.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return recency_ij.drop("recency", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/1234096421.py:14: FutureWarning: In a future version of pandas

In [84]:
features = ["price",
            "frequency_full", "frequency_l30", "frequency_l5",
            "frequency_category_full", "frequency_category_l30", "frequency_category_l5",
            "frequency_full_norm", "frequency_l30_norm", "frequency_l5_norm", 
            "coupon_frequency", "coupon_category_frequency",
            "bought_in_last_3_weeks", "bought_in_last_10_weeks",
            "coupon_recieved_last_week"
]
y = base_table_train["y"].values
X = base_table_train[features].values

In [85]:
log_reg = sklearn.linear_model.LogisticRegression(max_iter=300).fit(X, y)

In [86]:
log_reg.intercept_, log_reg.coef_

(array([-6.21811426]),
 array([[ 0.0029214 ,  2.39933514,  1.54041204, -0.13746636,  1.39713201,
          0.78728116, -0.91253288,  1.43433901,  0.26247722, -0.13705848,
         -0.11701984, -1.04559397,  0.75505432,  0.78485275,  1.32012272]]))

In [87]:
base_table_train["probability"] = log_reg.predict_proba(X)[:, 1]

In [88]:
score_yp(
    base_table_train["y"].values,
    base_table_train["probability"].values
)

0.0803272410603421

Validation: Test model performance (week 89)

In [89]:
base_table_validation = build_base_table(baskets, baskets_with_price, coupons, validation_week_5)

89
30
5
89


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_values except for the argument 'by' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)


30
5


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2261255535.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coupons_train["coupon_recieved"] = 1
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  base = build_target(baskets, week).drop("week", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/727766109.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return recency_ij.drop("recency", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/1234096421.py:14: FutureWarning: In a future version of pandas

In [90]:
features = ["price",
            "frequency_full", "frequency_l30", "frequency_l5",
            "frequency_category_full", "frequency_category_l30", "frequency_category_l5",
            "frequency_full_norm", "frequency_l30_norm", "frequency_l5_norm", 
            "coupon_frequency", "coupon_category_frequency",
            "bought_in_last_3_weeks", "bought_in_last_10_weeks",
            "coupon_recieved_last_week"
]
X_validation = base_table_validation[features].values

In [91]:
base_table_validation["probability"] = log_reg.predict_proba(X_validation)[:, 1]

In [92]:
score_yp(
    base_table_validation["y"].values,
    base_table_validation["probability"].values
)

0.07982302886092699

# Making the final prediction for week 90

In [93]:
base_table_test = build_base_table(baskets, baskets_with_price, coupons, test_week)

90
30
5
90


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_values except for the argument 'by' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/3640427484.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return purchase_frequency_ijc.sort_values(["customer", "product"], False).drop(["frequency_product", "frequency_category"], True)


30
5


C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/2210678271.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  base = build_target(baskets, week).drop("week", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/727766109.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return recency_ij.drop("recency", True)
C:\Users\Lodew\AppData\Local\Temp/ipykernel_17320/1234096421.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return df.drop("bought_in_last_3_weeks", True)


In [94]:
features = ["price",
            "frequency_full", "frequency_l30", "frequency_l5",
            "frequency_category_full", "frequency_category_l30", "frequency_category_l5",
            "frequency_full_norm", "frequency_l30_norm", "frequency_l5_norm", 
            "coupon_frequency", "coupon_category_frequency",
            "bought_in_last_3_weeks", "bought_in_last_10_weeks",
            "coupon_recieved_last_week"
]
X_test = base_table_test[features].values

In [95]:
base_table_test["probability"] = log_reg.predict_proba(X_test)[:, 1]

In [96]:
base_table_test

,week,customer,product,y,category,price,frequency_full,frequency_l30,frequency_l5,frequency_category_full,...,frequency_full_norm,frequency_l30_norm,frequency_l5_norm,coupon_frequency,coupon_category_frequency,coupon_frequency_norm,bought_in_last_3_weeks,bought_in_last_10_weeks,coupon_recieved_last_week,probability
0,90,0,0,0,0,0.0,0.000000,0.000000,0.0,0.088889,...,0.000000,0.00,0.0,0.000000,0.244444,0.000000,0.0,0.0,0,0.001838
1,90,0,1,0,0,0.0,0.000000,0.000000,0.0,0.088889,...,0.000000,0.00,0.0,0.022222,0.244444,0.090909,0.0,0.0,0,0.001834
2,90,0,2,0,0,0.0,0.000000,0.000000,0.0,0.088889,...,0.000000,0.00,0.0,0.033333,0.244444,0.136364,0.0,0.0,0,0.001831
3,90,0,3,0,0,0.0,0.000000,0.000000,0.0,0.088889,...,0.000000,0.00,0.0,0.022222,0.244444,0.090909,0.0,0.0,0,0.001834
4,90,0,4,0,0,620.0,0.088889,0.066667,0.0,0.088889,...,1.000000,1.00,0.0,0.033333,0.244444,0.136364,0.0,0.0,0,0.077495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,90,1999,245,0,24,549.0,0.133333,0.166667,0.2,0.622222,...,0.214286,0.25,0.5,0.044444,0.288889,0.153846,1.0,0.0,0,0.092481
499996,90,1999,246,0,24,0.0,0.000000,0.000000,0.0,0.622222,...,0.000000,0.00,0.0,0.033333,0.288889,0.115385,0.0,0.0,0,0.004091
499997,90,1999,247,0,24,0.0,0.000000,0.000000,0.0,0.622222,...,0.000000,0.00,0.0,0.033333,0.288889,0.115385,0.0,0.0,0,0.004091
499998,90,1999,248,0,24,0.0,0.000000,0.000000,0.0,0.622222,...,0.000000,0.00,0.0,0.022222,0.288889,0.076923,0.0,0.0,0,0.004096


# The output of the model for week 90

In [97]:
prediction["prediction"] = base_table_test["probability"]

In [98]:
prediction

,week,customer,product,prediction
0,90,0,0,0.001838
1,90,0,1,0.001834
2,90,0,2,0.001831
3,90,0,3,0.001834
4,90,0,4,0.077495
...,...,...,...,...
499995,90,1999,245,0.092481
499996,90,1999,246,0.004091
499997,90,1999,247,0.004091
499998,90,1999,248,0.004096


In [99]:
prediction.prediction.nunique()

223994

In [100]:
prediction["prediction"].max()

0.9650783497136256

In [101]:
prediction.to_parquet(f"{PATH_DATA}\\Assignment_2_470043js_predictions.parquet")